In [58]:
base_dir = r"C:\Users\wojci\OneDrive\Documents\My Games\Tabletop Simulator\Mods\Workshop"

In [59]:
key_phrase = "Witcher"

In [60]:
import os

f = []
for (dirpath, dirnames, filenames) in os.walk(base_dir):
    f.extend(filenames)
    break

jsons = [i for i in f if i[-5:] == ".json" and i != "WorkshopFileInfos.json"]


In [62]:
import json

mod_numerical_name = "none"

for json_file in jsons:
    # Opening JSON file
    f = open(os.path.join(base_dir, json_file), encoding="UTF-8")
    
    # returns JSON object as 
    # a dictionary
    data = json.load(f)

    # Closing file
    f.close()
    

    if key_phrase in data['GameMode']:
        print(data['GameMode'])
        mod_numerical_name = json_file
        print(json_file)
    

The Witcher: Old World
2988242008.json


In [86]:
import re
### Read whole as string and regex find all urls/ImageURL
with open(os.path.join(base_dir, mod_numerical_name), encoding="UTF-8") as f:
    file_text = f.read()

image_links = [i[1] for i in re.findall(r'("ImageURL": ")([^\"]+)', file_text)]

### TODO: ADD ASSET NICKNAME!!!!!

In [87]:
image_links

['http://cloud-3.steamusercontent.com/ugc/2033984369205358244/FB1BD9D9276923D86FABB2FBF3EB3D887435728D/',
 'http://cloud-3.steamusercontent.com/ugc/2033984369205435389/B66A599C439681F262CBA942F5995FEE3650F4B7/',
 'http://cloud-3.steamusercontent.com/ugc/2033984369208285532/9F07775DE9498032D249A2836A6173DADB079D6C/',
 'http://cloud-3.steamusercontent.com/ugc/2033984369208285532/9F07775DE9498032D249A2836A6173DADB079D6C/',
 'http://cloud-3.steamusercontent.com/ugc/2033984369208285532/9F07775DE9498032D249A2836A6173DADB079D6C/',
 'http://cloud-3.steamusercontent.com/ugc/2033984369208285532/9F07775DE9498032D249A2836A6173DADB079D6C/',
 'http://cloud-3.steamusercontent.com/ugc/2033984369208285532/9F07775DE9498032D249A2836A6173DADB079D6C/',
 'http://cloud-3.steamusercontent.com/ugc/2033984369208295983/522B35B50E97B7CA39A607DA9FF584B563CE75D9/',
 'http://cloud-3.steamusercontent.com/ugc/2033984369208295983/522B35B50E97B7CA39A607DA9FF584B563CE75D9/',
 'http://cloud-3.steamusercontent.com/ugc/2033